In [1]:
import pickle
import glob
import networkx as nx 
import matplotlib.pyplot as plt

to_classify = ["sample", "domain", "phylum", "class", "order", "family", "genus"]

disease = "ibs"
classifier = 'family'

In [4]:
files = glob.glob('results/'+disease+'/'+disease+'_dicts/*')
taxa_dicts = []

for file in files:
    with open(file, 'rb') as f:
        taxa_dict = pickle.load(f)
        taxa_dicts.append(taxa_dict)
        
# calculate thresholds

# find average value of each
avg_d = {}
total_organisms = {}
num_files = len(files)
num_files_occurred = {}

for file in files:
    with open(file, 'rb') as f:
        taxa_dict = pickle.load(f)
    
    # count total organisms per patient
    organism_count = 0
    for i in taxa_dict[classifier]:
        organism_count += taxa_dict[classifier][i]
        if i not in num_files_occurred:
            num_files_occurred[i] = 1
        else:
            num_files_occurred[i] += 1
    total_organisms[file] = organism_count

    # add normalized data to the total, and divide by number of patients to average
    for i in taxa_dict[classifier]:
        if i not in avg_d:
            avg_d[i] = taxa_dict[classifier][i] / (num_files * organism_count)
        else:
            avg_d[i] += taxa_dict[classifier][i] / (num_files * organism_count)

total_files_per_sample = {}

for file in files:
    with open(file, 'rb') as f:
        taxa_dict = pickle.load(f)
    for i in taxa_dict[classifier]:
        if taxa_dict[classifier][i] / total_organisms[file] < avg_d[i]:
            continue
        else:
            if i not in total_files_per_sample:
                total_files_per_sample[i] = {}
            for j in taxa_dict[classifier]:
                if taxa_dict[classifier][j] / total_organisms[file] >= avg_d[j]:
                    if j not in total_files_per_sample[i]:
                        total_files_per_sample[i][j] = 1
                    else:
                        total_files_per_sample[i][j] += 1   

In [5]:
# graphing

sum = 0
total = 0

for key in total_files_per_sample:
    for key2 in total_files_per_sample[key]:
        sum += total_files_per_sample[key][key2]
        total +=1

if total == 0:
    print("Error: no correlation")
avg_tfs = sum / total
print(avg_tfs, num_files)

graph = nx.Graph()
used = []
for node in total_files_per_sample:
    if node not in used:
        graph.add_node(node, node_size=0)
    used.append(node)    

for node in total_files_per_sample:
    for other in total_files_per_sample[node]:
        if node != other and total_files_per_sample[node][other] > avg_tfs * 2:             
            graph.add_edge(node, other, weight = 0.1 * (total_files_per_sample[node][other] / num_files), color = 'black')

16.736685440712147 87


In [ ]:
plt.figure(figsize=(15,15), dpi = 600)

nx.draw_circular(graph, node_size = 1, font_size = 25, width = 0.5)
# graph.remove_nodes_from(list(nx.isolates(graph)))
nx.draw(graph, node_size = 1, font_size = 25, width = 0.5)


plt.title("Graph of connections between " + classifier + " for disease " + disease.upper(), fontsize = 25)
plt.show()